# Auxiliando os Times de Marketing e Pricing da Empresa de Delivery

A gente trabalha em uma empresa de delivery, num time de dados centralizado que atende a diversas áreas e recebemos algumas demandas que serão respondidas por meio da Análise dos dados.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

Numa ação de marketing, para atrair mais entregadores, vamos dar uma bonificação para os 20 entregadores que possuem maior distância percorrida ao todo. A bonificação vai variar de acordo com o tipo de profissional que ele é e o modelo que ele usa para se locomover (moto, bike, etc). Levante essas informações.

In [2]:
# Lendo os dados em formato ".csv".
drivers = pd.read_csv('./data/drivers.csv')
deliveries = pd.read_csv('./data/deliveries.csv')
orders = pd.read_csv('./data/orders.csv')
stores = pd.read_csv('./data/stores.csv')
hubs = pd.read_csv('./data/hubs.csv')

In [3]:
# Filtrando somente as entregas realizadas.
deliveries = deliveries[deliveries['delivery_status'] == 'DELIVERED']

# Excluindo linhas com ids não informados.
deliveries = deliveries.dropna(subset=['driver_id'])

# Modificando os ids para ser do tipo int.
deliveries['driver_id'] = deliveries['driver_id'].astype(int)

In [4]:
# Filtrando os 20 entregadores com maior distância percorrida.
top20 = deliveries.groupby('driver_id')['delivery_distance_meters'].sum().sort_values(ascending=False).head(20)

# Juntando a tabela dos 20 entregadores com maior distância e suas informações de meio de transporte e tipo.
top20_infos = pd.merge(top20, drivers, on='driver_id')

# Foi possível ver que o meio de transporte de todos os 20 entregadores é a moto.
# Então basta separar pelos dois tipos de entregadores.
tops_operator = top20_infos[top20_infos['driver_type'] == 'LOGISTIC OPERATOR']
tops_freelance = top20_infos[top20_infos['driver_type'] == 'FREELANCE']

# Exportando tabelas
tops_operator.to_excel('./output/tops_operator.xlsx', index=False)
tops_freelance.to_excel('./output/tops_freelance.xlsx', index=False)

Além disso, o time de Pricing precisa ajustar os valores pagos aos entregadores. Para isso, eles precisam da distância média percorrida pelos entregadores separada por estado, já que cada região terá seu preço.

In [5]:
# Selecionando o estado de cada loja pelo estado do hub_id vinculado a ela.
hubs_stores = pd.merge(hubs[['hub_id', 'hub_state']], stores[['store_id', 'hub_id']], on='hub_id')

# Selecionando o identificador da entrega relacionada com o pedido e mantendo sempre o estado.
stores_orders = pd.merge(hubs_stores[['store_id', 'hub_state']], orders[['store_id', 'delivery_order_id', 'order_status']], on='store_id')

# Filtrando apenas as que o status do pedido estiver concluído.
stores_orders = stores_orders[stores_orders['order_status'] == 'FINISHED']

# Juntando o estado com os dados de id e distância de entregadores.
orders_delivery = pd.merge(stores_orders[['delivery_order_id', 'hub_state']], deliveries[['delivery_order_id', 'driver_id', 'delivery_distance_meters']], on='delivery_order_id')

# Tirando a coluna "delivery_order_id" que não é mais necessária.
orders_delivery = orders_delivery.drop('delivery_order_id', axis=1)

# Distância percorrida por cada entregador.
distancia_entregador = orders_delivery.groupby(['hub_state', 'driver_id'])['delivery_distance_meters'].mean()

# Transformando de Series para DataFrame.
distancia_entregador = distancia_entregador.reset_index()

# Distância média percorrida pelos entregadores separada por estado.
dados_estado = distancia_entregador.groupby('hub_state')['delivery_distance_meters'].mean()

# Transformando de Series para DataFrame.
dados_estado = dados_estado.reset_index()

In [6]:
# Aqui vou começar a construir gráficos usando matplotlib dos dados obtidos acima.

In [7]:
dados_estado

,hub_state,delivery_distance_meters
0,PR,2860.589616
1,RJ,2236.226318
2,RS,3232.776311
3,SP,3268.854052
